## Clone Repository

In [17]:
!git clone https://github.com/piainmaster/ProblematicInternetUse.git

Cloning into 'ProblematicInternetUse'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 7.40 KiB | 7.40 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [18]:
%cd ProblematicInternetUse

/content/ProblematicInternetUse/ProblematicInternetUse/ProblematicInternetUse


## Import Dataset

In [19]:
!pip install -q kaggle # install kaggle api client

In [23]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [24]:
!chmod 600 ~/.kaggle/kaggle.json

In [25]:
!kaggle competitions download -c child-mind-institute-problematic-internet-use

100% 6.20G/6.21G [01:15<00:00, 85.4MB/s]
100% 6.21G/6.21G [01:15<00:00, 88.3MB/s]


In [26]:
!unzip -q child-mind-institute-problematic-internet-use.zip -d /content/dataset

## Push changes to GitHub

In [27]:
!git config --global user.email "pia.neuwirth@stud.plus.ac.at"
!git config --global user.name "piainmaster"

In [28]:
!git add Main.ipynb
!git commit -m "Updated the notebook with new changes"

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	child-mind-institute-problematic-internet-use.zip

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
!git push origin main

## Import Data

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

CSV DATA

In [3]:
# Import data sets
train_data = pd.read_csv("/content/dataset/train.csv")
test_data = pd.read_csv("/content/dataset/test.csv")

print("Train data shape:" + str(train_data.shape))
# Check for missing values
print("\nMissing Values in Train Data:")
print(train_data.isnull().sum())

Train data shape:(3960, 82)

Missing Values in Train Data:
id                                           0
Basic_Demos-Enroll_Season                    0
Basic_Demos-Age                              0
Basic_Demos-Sex                              0
CGAS-Season                               1405
                                          ... 
SDS-SDS_Total_Raw                         1351
SDS-SDS_Total_T                           1354
PreInt_EduHx-Season                        420
PreInt_EduHx-computerinternet_hoursday     659
sii                                       1224
Length: 82, dtype: int64


Parquet Data

In [2]:
dataset_folder = '/content/dataset/series_train.parquet'

# Get a list of all subfolders (which contain "id=xxxxx")
subfolders = [f.path for f in os.scandir(dataset_folder) if f.is_dir()]
chunk_size = len(subfolders) // 9
subfolders_parts = [subfolders[i:i + chunk_size] for i in range(0, len(subfolders), chunk_size)]

# If the last chunk is smaller than expected (due to uneven split), it will still work.
print(f"Number of subfolders in each part (except possibly the last): {chunk_size}")
print(f"Number of parts created: {len(subfolders_parts)}")

Number of subfolders in each part (except possibly the last): 110
Number of parts created: 10


In [4]:
import pandas as pd

# Initialize an empty list to hold DataFrame chunks for each part
full_df_parts = []

# Process the first 5 parts
for idx in range(5):
    subfolders_part = subfolders_parts[idx]  # Get the current part
    dataframes_part = []  # Temporary list for the current part's DataFrames

    print(f"Processing part {idx + 1}/5...")

    # Loop through each subfolder in the current part and read the `.parquet` file
    for subfolder in subfolders_part:
        parquet_file = os.path.join(subfolder, 'part-0.parquet')
        df_chunk = pd.read_parquet(parquet_file)
        dataframes_part.append(df_chunk)

    part_df = pd.concat(dataframes_part, ignore_index=True)
    full_df_parts.append(part_df)
    print(f"Part {idx + 1} loaded and concatenated, shape: {part_df.shape}")

# Process the last 5 parts
for idx in range(5, 10):
    subfolders_part = subfolders_parts[idx]  # Get the current part
    dataframes_part = []  # Temporary list for the current part's DataFrames

    print(f"Processing part {idx + 1}/10...")

    # Loop through each subfolder in the current part and read the `.parquet` file
    for subfolder in subfolders_part:
        parquet_file = os.path.join(subfolder, 'series_train.parquet')

        df_chunk = pd.read_parquet(parquet_file)
        dataframes_part.append(df_chunk)

    part_df = pd.concat(dataframes_part, ignore_index=True)
    full_df_parts.append(part_df)

    print(f"Part {idx + 1} loaded and concatenated, shape: {part_df.shape}")

# Concatenate all parts into the final full DataFrame
full_df = pd.concat(full_df_parts, ignore_index=True)

# Check the final DataFrame
print(f"Full DataFrame Shape: {full_df.shape}")
full_df.head()

Processing part 1/5...
Part 1 loaded and concatenated, shape: (34848507, 13)
Processing part 2/5...
Part 2 loaded and concatenated, shape: (35791579, 13)
Processing part 3/5...
Part 3 loaded and concatenated, shape: (35509495, 13)
Processing part 4/5...
Part 4 loaded and concatenated, shape: (35216892, 13)
Processing part 5/5...
Part 5 loaded and concatenated, shape: (35088228, 13)
Processing part 6/10...


FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset/series_train.parquet/id=66f619e3/series_train.parquet'

In [7]:
# Import data sets
#train_data = pd.read_parquet("/content/dataset/series_train.parquet")
test_data = pd.read_parquet("/content/dataset/series_test.parquet")

df = test_data
df.to_csv('series_test.csv')

print("Test data shape:" + str(df.shape))
# Check for missing values
print("\nMissing Values in Test Data:")
print(df.isnull().sum())

print(df.head())
print(df.info())

Test data shape:(439726, 14)

Missing Values in Test Data:
step                   0
X                      0
Y                      0
Z                      0
enmo                   0
anglez                 0
non-wear_flag          0
light                  0
battery_voltage        0
time_of_day            0
weekday                0
quarter                0
relative_date_PCIAT    0
id                     0
dtype: int64
   step         X         Y         Z      enmo     anglez  non-wear_flag  \
0     0  0.021536  0.022214 -1.022370  0.022853 -88.280762            0.0   
1     1  0.022005  0.022187 -1.019740  0.020231 -88.241707            0.0   
2     2  0.022240  0.022005 -1.019401  0.019893 -88.170067            0.0   
3     3  0.021589  0.022578 -1.018177  0.018667 -88.250031            0.0   
4     4  0.022005  0.023763 -1.014323  0.016848 -88.130775            0.0   

       light  battery_voltage     time_of_day  weekday  quarter  \
0  53.000000      4188.000000  56940000000000   